# 🔧 02_Preprocessing — 전처리 및 피처 엔지니어링

## 목적
- SDTM 도메인 간 USUBJID 기준 병합
- 도메인 지식 기반 파생 변수(Feature) 생성
- 결측치 처리 및 인코딩
- Data Leakage 검토
- 최종 분석 데이터셋(model_dataset.csv) 생성

## 1. 환경 설정 및 데이터 로드

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Load all domains
dm = pd.read_sas('../data/raw/dm.xpt', format='xport', encoding='utf-8')
ae = pd.read_sas('../data/raw/ae.xpt', format='xport', encoding='utf-8')
ds = pd.read_sas('../data/raw/ds.xpt', format='xport', encoding='utf-8')
sv = pd.read_sas('../data/raw/sv.xpt', format='xport', encoding='utf-8')
vs = pd.read_sas('../data/raw/vs.xpt', format='xport', encoding='utf-8')
cm = pd.read_sas('../data/raw/cm.xpt', format='xport', encoding='utf-8')
mh = pd.read_sas('../data/raw/mh.xpt', format='xport', encoding='utf-8')

# Screen Failure 제외
dm_enrolled = dm[dm['ARM'] != 'Screen Failure'].copy()
enrolled_ids = set(dm_enrolled['USUBJID'])
print(f"분석 대상: {len(enrolled_ids)}명 (Screen Failure 제외)")

## 2. 타겟 변수 생성
DS 도메인의 DISPOSITION EVENT에서 COMPLETED 여부로 이진 분류 타겟 생성

In [ ]:
disp = ds[ds['DSCAT'] == 'DISPOSITION EVENT'].copy()
disp = disp[disp['USUBJID'].isin(enrolled_ids)]
disp['DROPOUT'] = (disp['DSDECOD'] != 'COMPLETED').astype(int)
target = disp[['USUBJID', 'DROPOUT']].copy()

print(f"Completed: {(target['DROPOUT']==0).sum()} ({(target['DROPOUT']==0).mean()*100:.1f}%)")
print(f"Dropout:   {(target['DROPOUT']==1).sum()} ({(target['DROPOUT']==1).mean()*100:.1f}%)")

## 3. DM 피처 — 인구통계
**도메인 지식 근거**: 나이, 성별, 투약군은 탈락에 영향을 미치는 기본 요인

In [ ]:
dm_features = dm_enrolled[['USUBJID', 'AGE', 'SEX', 'RACE', 'ARM', 'SITEID']].copy()

# 인코딩
dm_features['SEX_N'] = dm_features['SEX'].map({'F': 0, 'M': 1})
dm_features['RACE_WHITE'] = (dm_features['RACE'] == 'WHITE').astype(int)

# ARM → One-Hot (투약군은 탈락의 핵심 요인)
dm_features['ARM_PLACEBO'] = (dm_features['ARM'] == 'Placebo').astype(int)
dm_features['ARM_LOW'] = (dm_features['ARM'] == 'Xanomeline Low Dose').astype(int)
dm_features['ARM_HIGH'] = (dm_features['ARM'] == 'Xanomeline High Dose').astype(int)

print(f"DM features: {dm_features.shape}")
dm_features.head()

## 4. AE 피처 — 이상반응
**도메인 지식 근거**: 
- 이 시험의 탈락 사유 1위가 Adverse Event (63.9%)
- AE의 건수뿐 아니라 심각도, 약물 관련성, 회복 여부가 탈락 결정에 영향
- 패치 제형이므로 피부 관련 AE가 특히 중요

In [ ]:
ae_enrolled = ae[ae['USUBJID'].isin(enrolled_ids)]
ae_features = pd.DataFrame({'USUBJID': list(enrolled_ids)})

# 총 AE 건수
ae_count = ae_enrolled.groupby('USUBJID').size().reset_index(name='AE_COUNT')
ae_features = ae_features.merge(ae_count, on='USUBJID', how='left')

# SAE 경험 여부
ae_sae = ae_enrolled[ae_enrolled['AESER']=='Y'].groupby('USUBJID').size().reset_index(name='SAE_COUNT')
ae_features = ae_features.merge(ae_sae, on='USUBJID', how='left')
ae_features['HAS_SAE'] = (ae_features['SAE_COUNT'] > 0).astype(int)

# 심각도별 건수
for sev in ['MILD', 'MODERATE', 'SEVERE']:
    temp = ae_enrolled[ae_enrolled['AESEV']==sev].groupby('USUBJID').size().reset_index(name=f'AE_{sev}')
    ae_features = ae_features.merge(temp, on='USUBJID', how='left')

# 약물 관련 AE (PROBABLE/POSSIBLE)
ae_related = ae_enrolled[ae_enrolled['AEREL'].isin(['PROBABLE','POSSIBLE'])]
ae_features = ae_features.merge(
    ae_related.groupby('USUBJID').size().reset_index(name='AE_RELATED'), on='USUBJID', how='left')

# 미회복 AE
ae_unres = ae_enrolled[ae_enrolled['AEOUT']=='NOT RECOVERED/NOT RESOLVED']
ae_features = ae_features.merge(
    ae_unres.groupby('USUBJID').size().reset_index(name='AE_UNRESOLVED'), on='USUBJID', how='left')

# Body System 다양성
ae_features = ae_features.merge(
    ae_enrolled.groupby('USUBJID')['AEBODSYS'].nunique().reset_index(name='AE_BODSYS_N'), on='USUBJID', how='left')

# 피부 관련 AE (패치 제형 특성)
skin_ae = ae_enrolled[ae_enrolled['AEBODSYS'].isin([
    'SKIN AND SUBCUTANEOUS TISSUE DISORDERS',
    'GENERAL DISORDERS AND ADMINISTRATION SITE CONDITIONS'])]
ae_features = ae_features.merge(
    skin_ae.groupby('USUBJID').size().reset_index(name='AE_SKIN_COUNT'), on='USUBJID', how='left')

ae_features = ae_features.drop(columns=['SAE_COUNT']).fillna(0)
print(f"AE features: {ae_features.shape}")
ae_features.describe().round(2)

## 5. SV 피처 — 방문 기록
**⚠️ Data Leakage 주의**: 방문 관련 피처는 탈락 "후"에 확정되므로 모델링 시 제거 필요

In [ ]:
sv_enrolled = sv[sv['USUBJID'].isin(enrolled_ids)]
sv_features = pd.DataFrame({'USUBJID': list(enrolled_ids)})

# 총/정규/비정규 방문 횟수
sv_features = sv_features.merge(
    sv_enrolled.groupby('USUBJID').size().reset_index(name='VISIT_TOTAL'), on='USUBJID', how='left')

sv_sched = sv_enrolled[sv_enrolled['VISIT'].str.contains('WEEK|BASELINE|SCREENING', na=False)]
sv_features = sv_features.merge(
    sv_sched.groupby('USUBJID').size().reset_index(name='VISIT_SCHEDULED'), on='USUBJID', how='left')

sv_unsched = sv_enrolled[sv_enrolled['VISIT'].str.contains('UNSCHEDULED', na=False)]
sv_features = sv_features.merge(
    sv_unsched.groupby('USUBJID').size().reset_index(name='VISIT_UNSCHEDULED'), on='USUBJID', how='left')

# 최대 도달 WEEK
week_vis = sv_enrolled[sv_enrolled['VISIT'].str.contains('WEEK', na=False)].copy()
week_vis['WEEK_NUM'] = week_vis['VISIT'].str.extract(r'WEEK\s+(\d+)').astype(float)
sv_features = sv_features.merge(
    week_vis.groupby('USUBJID')['WEEK_NUM'].max().reset_index(name='MAX_WEEK'), on='USUBJID', how='left')

sv_features = sv_features.fillna(0)
print(f"SV features: {sv_features.shape}")
print("⚠️ VISIT_TOTAL, VISIT_SCHEDULED, MAX_WEEK 등은 Data Leakage → 모델링에서 제거 예정")

## 6. VS 피처 — 활력징후 (Baseline)

In [ ]:
vs_enrolled = vs[vs['USUBJID'].isin(enrolled_ids)]
vs_features = pd.DataFrame({'USUBJID': list(enrolled_ids)})

# Baseline 값 추출
vs_bl = vs_enrolled[vs_enrolled['VSBLFL']=='Y']
for testcd in ['SYSBP', 'DIABP', 'PULSE', 'TEMP', 'WEIGHT']:
    temp = vs_bl[vs_bl['VSTESTCD']==testcd][['USUBJID','VSSTRESN']]
    temp = temp.groupby('USUBJID')['VSSTRESN'].mean().reset_index(name=f'VS_BL_{testcd}')
    vs_features = vs_features.merge(temp, on='USUBJID', how='left')

# 혈압/맥박 변동성 (표준편차)
vs_features = vs_features.merge(
    vs_enrolled[vs_enrolled['VSTESTCD']=='SYSBP'].groupby('USUBJID')['VSSTRESN'].std().reset_index(name='VS_SYSBP_STD'), 
    on='USUBJID', how='left')
vs_features = vs_features.merge(
    vs_enrolled[vs_enrolled['VSTESTCD']=='PULSE'].groupby('USUBJID')['VSSTRESN'].std().reset_index(name='VS_PULSE_STD'),
    on='USUBJID', how='left')

print(f"VS features: {vs_features.shape}")
vs_features.describe().round(2)

## 7. CM & MH 피처 — 병용약물 및 병력

In [ ]:
# CM Features
cm_enrolled = cm[cm['USUBJID'].isin(enrolled_ids)]
cm_features = pd.DataFrame({'USUBJID': list(enrolled_ids)})
cm_features = cm_features.merge(
    cm_enrolled.groupby('USUBJID').size().reset_index(name='CM_COUNT'), on='USUBJID', how='left')
cm_features = cm_features.merge(
    cm_enrolled.groupby('USUBJID')['CMDECOD'].nunique().reset_index(name='CM_UNIQUE'), on='USUBJID', how='left')
cm_features = cm_features.merge(
    cm_enrolled[cm_enrolled['CMCLAS']!='UNCODED'].groupby('USUBJID')['CMCLAS'].nunique().reset_index(name='CM_CLASS_N'),
    on='USUBJID', how='left')
cm_features = cm_features.fillna(0)

# MH Features
mh_enrolled = mh[mh['USUBJID'].isin(enrolled_ids)]
mh_features = pd.DataFrame({'USUBJID': list(enrolled_ids)})
mh_features = mh_features.merge(
    mh_enrolled.groupby('USUBJID').size().reset_index(name='MH_COUNT'), on='USUBJID', how='left')
mh_features = mh_features.merge(
    mh_enrolled[mh_enrolled['MHCAT']=='SIGNIFICANT PRE-EXISTING CONDITION'].groupby('USUBJID').size().reset_index(name='MH_PREEXIST'),
    on='USUBJID', how='left')
mh_features = mh_features.merge(
    mh_enrolled.groupby('USUBJID')['MHBODSYS'].nunique().reset_index(name='MH_BODSYS_N'), on='USUBJID', how='left')
mh_hyp = mh_enrolled[mh_enrolled['MHDECOD']=='HYPERTENSION'].groupby('USUBJID').size().reset_index(name='MH_HYPERTENSION')
mh_features = mh_features.merge(mh_hyp, on='USUBJID', how='left')
mh_features['MH_HYPERTENSION'] = (mh_features['MH_HYPERTENSION'] > 0).astype(int)
mh_features = mh_features.fillna(0)

print(f"CM features: {cm_features.shape}")
print(f"MH features: {mh_features.shape}")

## 8. 전체 병합 및 최종 데이터셋

In [ ]:
# Merge all
df = target.copy()
for feat_df in [dm_features, ae_features, sv_features, vs_features, cm_features, mh_features]:
    df = df.merge(feat_df, on='USUBJID', how='left')

# 불필요한 원본 컬럼 제거
df = df.drop(columns=['SEX', 'RACE', 'ARM', 'SITEID'], errors='ignore')

# 결측치 처리 (중앙값)
numeric_cols = df.select_dtypes(include=[np.number]).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

print(f"최종 데이터셋: {df.shape}")
print(f"타겟: Completed={sum(df['DROPOUT']==0)}, Dropout={sum(df['DROPOUT']==1)}")
print(f"결측치: {df.isnull().sum().sum()}")

# 저장
df.to_csv('../data/model_dataset.csv', index=False)
print("\n✅ model_dataset.csv 저장 완료")
df.head()

## 9. 상관관계 분석 & Data Leakage 확인

In [ ]:
features = [c for c in df.columns if c not in ['USUBJID', 'DROPOUT']]
corr_with_target = df[features + ['DROPOUT']].corr()['DROPOUT'].drop('DROPOUT').sort_values(key=abs, ascending=False)

print("📌 DROPOUT과의 상관관계 (절대값 순):")
for col, val in corr_with_target.head(15).items():
    sign = "+" if val > 0 else "-"
    flag = " ⚠️ LEAKAGE" if abs(val) > 0.7 else ""
    print(f"  {sign} {col}: {val:.3f}{flag}")

print("\n⚠️ VISIT_TOTAL, VISIT_SCHEDULED, MAX_WEEK은 탈락 '결과'로 인한 상관관계 → 모델링에서 제거")